In [ ]:
#SE REALIZAN LAS IMPORTACIONES NECESARIAS
%matplotlib inline
from pymongo import MongoClient
import bson
from bson import json_util
import pandas as pd
import datetime
import time
from datetime import datetime as dt
import json
import re
import os
from bson.json_util import dumps
from pandas.io.json import json_normalize
import xlsxwriter
import matplotlib
import matplotlib.pyplot as plt
import pandas_profiling

In [ ]:
#CONEXION A LA BASE DE DATOS
ip = 'obfuscated'
port = obfuscated

client = MongoClient(ip, port)
db = client['obfuscated']

# LLAMADO A LA COLECCION ManualClassificationValues
manualClassValues = db['obfuscated']

# LLAMADO A LA COLECCION ScraperValues
scraperValues = db['obfuscated']

# LLAMADO A LA COLECCION JoinedVulnerabilitiesDateFormat
joinedVulnerabilitiesDateFormat = db['obfuscated']

## I. Procedimiento para leer el archivo de clasificación manual (.xlsx) y cargarlo a MongoDB

In [ ]:
#SE IMPORTA EL ARCHIVO
xls_file = pd.ExcelFile('/obfuscated.xlsx')

# SE IMPRIME EL NOMBRE DE LAS HOJAS DE CALCULO PARA SABER CUAL ES LA DE INTERES
# xls_file.sheet_names

# SE DEFINE UN NUEVO DATAFRAME CON LA HOJA DE CALCULO DE INTERES
sheetName = 'obfuscated'
df = xls_file.parse(sheetName)
# SE ELIMINA LA COLUMNA NUMBER YA QUE NO ES RELEVANTE
del df['Number']

In [ ]:
# SE DEFINE UN METODO PARA EXTRAER UN CVE-ID DE UN STRING POR MEDIO DE EXPRESIONES REGULARES
def getID(cveLink):
    found = re.search('CVE-[0-9]+-[0-9]+', cveLink).group(0)
    return found

In [ ]:
# SE CREA UNA NUEVA COLUMNA LLAMADA ID, EN LA CUAL SE GUARDA EL ID EXTRAIDO DE LA COLUMNA CVE-LINK
df['cveId'] = df['CVE-link'].map(getID)

In [ ]:
#SE HACE LA CONVERSION DEL DATFRAME A JSON
exportJSON = df.to_json(orient='records')

#SE GUARDA EL JSON EN UN ARCHIVO CON EXTENSION .json
with open('obfuscated.json', 'w') as file:
     json.dump(exportJSON, file)

In [ ]:
#AGREGAR LOS DOCUMENTOS A UNA COLECCION, SI LA COLECCION NO TIENE DOCUMENTOS SE AGREGAN 
#DE LO CONTRARIO SE BORRAN LOS QUE ESTABAN Y SE AGREGAN LOS NUEVOS
def addToCollection(collection, documents):
    if collection.count() == 0:
        data = json_util.loads(documents)
        collection.insert_many(data)
    else:
        collection.delete_many({})
        data = json_util.loads(documents)
        collection.insert_many(data)

In [ ]:
# INVOCACION AL METODO addToCollection() PARA GUARDAR EL DATAFRAME CONVERTIDO A JSON
addToCollection(manualClassValues, exportJSON)

## II. Procedimiento para leer archivos de la información extraida con web scraping (.json) del web scraping y cargarlos a MongoDB

In [ ]:
#METODO PARA SABER SI UNA COLLECCION ESTA VACIA
def isCollectionEmpty(collection):
    if collection.count() == 0:
        return True
    else:
        return False

In [ ]:
#ENTRAR A UNA RUTA Y LEER CADA ARCHIVO .json QUE AHI SE ENCUENTRA, SE LEE SU CONTENIDO Y SE AGREGA UNO A UNO A MONGO
path_to_jsons = '/obfuscated/CVE-DATA'
data = {}

if isCollectionEmpty(scraperValues):
    for pos_json in os.listdir(path_to_jsons):
        if pos_json.endswith('.json'):
            f = open(os.path.join(path_to_jsons, pos_json), 'r')
            data = json.load(f)
            scraperValues.insert_one(data)
else:
    scraperValues.delete_many({})
    for pos_json in os.listdir(path_to_jsons):
        if pos_json.endswith('.json'):
            f = open(os.path.join(path_to_jsons, pos_json), 'r')
            data = json.load(f)
            scraperValues.insert_one(data)

## III. Extraer información de Mongo según rango de fechas (Informacion completa)

In [ ]:
years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
semesters = [1, 2]

In [ ]:
def getDataByDateRange(year, semester):
    
    if semester == 1:
            return joinedVulnerabilitiesDateFormat.aggregate(
            [

            { "$match": 
                {
                    "webScrapingInformation.publishDate": 
                    {
                        "$gte": dt(year,1,1), 
                        "$lte": dt(year,6,30)
                    }
                }
            },
            { "$sort": { "webScrapingInformation.publishDate": 1 } },
            { "$project" : 
                { 
                    "_id" : 0,
                    "cveId" : 1,
                    "Manually assigned type (high-level)" : 1,
                    "Manually assigned type (low-level)" : 1,
                    "high-level" : 1,
                    "subsystem" : 1,
                    "publishDate" : { "$dateToString": { "format": "%Y-%m-%d", "date": "$webScrapingInformation.publishDate" } },
                    "range": {"$literal": str(year)+"-"+str(semester)}
            
                }
            }
        
            
            ]
            )
        
    if semester == 2:
            return joinedVulnerabilitiesDateFormat.aggregate(
            [

            { "$match": 
                {
                    "webScrapingInformation.publishDate": 
                    {
                        "$gte": dt(year,7,1), 
                        "$lte": dt(year,12,31)
                    }
                }
            },
            { "$sort": { "webScrapingInformation.publishDate": 1 } },
            { "$project" : 
                { 
                    "_id" : 0,
                    "cveId" : 1,
                    "Manually assigned type (high-level)" : 1,
                    "Manually assigned type (low-level)" : 1,
                    "high-level" : 1,
                    "subsystem" : 1,
                    "publishDate" : { "$dateToString": { "format": "%Y-%m-%d", "date": "$webScrapingInformation.publishDate" } },
                    "range": {"$literal": str(year)+"-"+str(semester)}
            
                }
            }
            ]
            )


In [ ]:
def writeToExcel():
    # SE CREA UN Pandas Excel writer USANDO XlsxWriter
    writer = pd.ExcelWriter('vulnerabilites_semesters.xlsx', engine='xlsxwriter')
    # SE HACE UN ITERACION A LOS ANOS Y A LOS SEMESTRES, DEVUELVE LA INFORMACION POR CADA PERIODO Y LA GUARDA EN UNA
    # PESTAÑA DIFERENTE POR PERIODO EN EL EXCEL DESTINO
    
    for year in years:
        for semester in semesters:
            JSONRangeDate = getDataByDateRange(year, semester)
            dataFromJSONRangeDate = json.loads(dumps(JSONRangeDate))     
            dataFrame = json_normalize(dataFromJSONRangeDate)
            dataFrame.to_excel(writer, sheet_name=""+str(year)+"-"+str(semester))
        
    writer.save()

writeToExcel()

In [ ]:
def getDataFrameAggregateInfo():
    dataFrame = pd.DataFrame([])
    
    for year in years:
        for semester in semesters:
            JSONRangeDate = getDataByDateRange(year, semester)
            dataFromJSONRangeDate = json.loads(dumps(JSONRangeDate))  
            dataFrameTemp = json_normalize(dataFromJSONRangeDate)
            dataFrame = dataFrame.append(pd.DataFrame(dataFrameTemp))
    return dataFrame

In [ ]:
dataFrameAggregateInfo = getDataFrameAggregateInfo()
pandas_profiling.ProfileReport(dataFrameAggregateInfo)

## IV. Extraer información de Mongo según rango de fechas (Count)

In [ ]:
def getDataCountByDateRange(year, semester):
    
    if semester == 1:
            return joinedVulnerabilitiesDateFormat.aggregate(
            [

            { "$match": 
                {
                    "webScrapingInformation.publishDate": 
                    {
                        "$gte": dt(year,1,1), 
                        "$lte": dt(year,6,30)
                    }
                }
            },
            { "$sort": { "webScrapingInformation.publishDate": 1 } },
            { "$project" : 
                { 
                    "_id" : 0,
                    "cveId" : 1,
                    "Manually assigned type (high-level)" : 1,
                    "Manually assigned type (low-level)" : 1,
                    "high-level" : 1,
                    "subsystem" : 1,
                    "publishDate" : { "$dateToString": { "format": "%Y-%m-%d", "date": "$webScrapingInformation.publishDate" } },
                    "range": {"$literal": str(year)+"-"+str(semester)}
            
                }
            },
            {
              "$count": "count"
            }
            ]
            )
        
    if semester == 2:
            return joinedVulnerabilitiesDateFormat.aggregate(
            [

            { "$match": 
                {
                    "webScrapingInformation.publishDate": 
                    {
                        "$gte": dt(year,7,1), 
                        "$lte": dt(year,12,31)
                    }
                }
            },
            { "$sort": { "webScrapingInformation.publishDate": 1 } },
            { "$project" : 
                { 
                    "_id" : 0,
                    "cveId" : 1,
                    "Manually assigned type (high-level)" : 1,
                    "Manually assigned type (low-level)" : 1,
                    "high-level" : 1,
                    "subsystem" : 1,
                    "publishDate" : { "$dateToString": { "format": "%Y-%m-%d", "date": "$webScrapingInformation.publishDate" } },
                    "range": {"$literal": str(year)+"-"+str(semester)}
            
                }
            },
            {
              "$count": "count"
            }
            ]
            )


In [ ]:
def getJSONCount():

    dataCount = []
    for year in years:
        for semester in semesters:
            response_json = {}
            response_json["semester"] = str(year)+"-"+str(semester)
            response_json["total"] = getDataCountByDateRange(year, semester)
            dataCount.append(response_json)
    dataFromJSONRangeDate = json.loads(dumps(dataCount))     
    return json_normalize(dataFromJSONRangeDate)

dataframeSemesterTotal = getJSONCount()

#NORMALIZA LA COLUMNA TOTAL DEVOLVIENDO EL ENTERO CORRESPONDIENTE
def getCount(total):
    if total == []:
        return 0
    else:
        return total[0]['count']

# LA COLUMNA TOTAL SE MODIFICA PARA QUE TENGA EL VALOR ENTERO DE LA CANTIDAD DE VULNERABILIDADES POR SEMESTRE
dataframeSemesterTotal['total'] = dataframeSemesterTotal['total'].map(getCount)

In [ ]:
a = dataframeSemesterTotal.plot(x="semester", kind='bar', legend=False, title='Vulnerabilities over time', rot=0, figsize=(15, 6))
a.set_xlabel('Semesters')
a.set_ylabel('Vulnerabilities')

for p in a.patches: 
    a.annotate(p.get_height(), (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 7), textcoords='offset points')

plt.gcf().subplots_adjust(bottom=0.15)
plt.savefig('VulnerabilitiesSemesters.png', bbox_inches='tight')